In [1]:
import os
%pwd

'c:\\Users\\meabh\\VS Code Projects\\Kidney-Tumor-classification-DL-using-MLFlow-CICD\\research'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\meabh\\VS Code Projects\\Kidney-Tumor-classification-DL-using-MLFlow-CICD'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


## this code is an Entity

# it is combination of config+params (.yaml)

In [4]:
from cnnClassifier.constants import *       # In the constants folder, there root file path is given
from cnnClassifier.utils.common import read_yaml, create_directories   # In the utils folder, we have imported required function
import tensorflow as tf

## This is configuration ralated things

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])     # It will create a folder

    
    def get_training_config(self) -> TrainingConfig:            ## here we are going to give all the Path
        training = self.config.training                          
        prepare_base_model = self.config.prepare_base_model     # this is updated base model path
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Test2")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [6]:
## Now prepare the components

In [7]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [8]:
class Training:
    def __init__(self, config: TrainingConfig):    # this class will take input from class defined above
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    ## This function will load the model from the path - updated_base_model_path

    def train_valid_generator(self):         ## This is the keras data generator function used for generating a dataset like train, test, split, and more.
                                             ## This code can get from keras documentation, just search for "train_valid_generator" function.

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    # Now define a function for saving the model

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):   # This function will save the model. It will take the model & save the model
        model.save(path)


    # Now lets define the training function


    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

    

In [9]:
## update the Pipeline

In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2023-11-26 14:08:00,007: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-26 14:08:00,011: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-26 14:08:00,013: INFO: common: created directory at: artifacts]
[2023-11-26 14:08:00,015: INFO: common: created directory at: artifacts\training]


OSError: No file or directory found at artifacts\prepare_base_model\base_model_updated.h5